In [1]:
import os
import json
import re

In [2]:
def merge_details(details):
    merged_list = []
    current_item = ""

    for i, item in enumerate(details):
        if item.strip() and item.strip()[0].isdigit():
            j = 1
            while j < len(item.strip()) and item.strip()[j].isdigit():
                j += 1
            if j < len(item.strip()) and item.strip()[j] == "：":
                merged_list.append(current_item)
                current_item = item.strip()
            else:
                current_item += item.strip()
        else:
            current_item += item.strip()

    merged_list.append(current_item)
    merged_list.pop(0)
    return merged_list


def extract_tuples(string):
    tuples = []
    depth = 0
    start = None
    for i, char in enumerate(string):
        if char == "(" or char == "（":
            if depth == 0:
                start = i
            depth += 1
        elif char == ")" or char == "）":
            depth -= 1
            if depth == 0 and start is not None:
                tuples.append(string[start+1:i])
                start = None
    return tuples


def get_results(merged_list, input_list, entity_type_dict, ent_extend_map, mode):
    # input_list2ent = {}
    # for idx, item in enumerate(input_list):
    #     input_list2ent[item] = entity_type_list[idx//10]

    ent_extend_index = 0
    cur_ent_extend_list = []
    A = []

    for idx, item in enumerate(merged_list):
        B = []
        if 'en' in mode and "``Text : " in item:
            item = item.split("``Text : ")[0]
        cur_output_3 = item.split('Output:')
        
        cur_idx = cur_output_3[0].split('：')[0]
        # cur_example = input_list[int(cur_idx)-1]
        # cur_true_ent = input_list2ent[cur_example]
        if 'zh' in mode:
            cur_true_ent = entity_type_dict[input_list[int(cur_idx)-1]['ent_type']]
        elif 'en' in mode:
            cur_true_ent = input_list[int(cur_idx)-1]['ent_type']
            
        if ent_extend_index == len(cur_ent_extend_list):
            ent_extend_index = 0

        cur_ent_extend_list = ent_extend_map[cur_true_ent]
        cur_extend_ent = cur_ent_extend_list[ent_extend_index]

        for output in cur_output_3[1:]:
            sep = ""
            if 'zh' in mode:
                if "生成实体列表：" in output:
                    sep = "生成实体列表："
            elif 'en' in mode:
                
                if "Entity list:" in output:
                    sep = "Entity list:"
                elif "Generate an entity list:" in output:
                    sep = "Generate an entity list:"
                elif "Generated entity list:" in output:
                    sep = "Generated entity list:"
            if sep:
                tmp = output.split(sep)[1]
                matches = extract_tuples(tmp)
                if matches:
                    for match in matches:
                        if 'zh' in mode:
                            triple = tuple(match.strip().strip('（）').split(','))
                        elif 'en' in mode:
                            triple = tuple(match.strip().strip('()').split(','))
                        if len(triple) == 2:
                            B.append((int(cur_idx)-1, cur_extend_ent, triple[1].strip()))
                        else:
                            B.append((int(cur_idx)-1, cur_extend_ent, ''))
                else:
                    B.append((int(cur_idx)-1, cur_extend_ent, ''))
                
            else:
                B.append((int(cur_idx)-1, cur_extend_ent, ''))
                continue
        if B:
            A.append(B)    

        ent_extend_index += 1

    A = [list(set(item)) for item in A]
    
    with open(f'./results/test_{mode}.txt','w',encoding='utf-8') as f:
        for item in A:
            f.write(str(item)+'\n')

CMeEE

In [15]:
with open('./results/details_zh.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/cmeee/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/cmeee/ent_extend_map.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

entity_type_dict = {
            'dru':'药物',
            'bod':'身体',
            'pro':'医疗程序',
            'sym':'临床表现',
            'equ':'医疗设备',
            'ite':'医学检验项目',
            'dep':'科室',
            'mic':'微生物类',
            'dis':'疾病'
}
merged_list = merge_details(test_details)
get_results(merged_list, input_list, entity_type_dict, ent_extend_map, 'zh')


ACE05

In [21]:
with open('./results/details_en.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/ace05/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/ace05/ent_extend_map.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)


merged_list = merge_details(test_details)
get_results(merged_list, input_list, None, ent_extend_map, 'en')

Baichuan2-CMeEE

In [3]:
with open('./results/details_baichuan2_13B_zh.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/cmeee/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/cmeee/final_entity_extend_map_baichuan2_13B_zh.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

# entity_type_list = ['dru', 'bod', 'pro', 'sym', 'equ', 'ite', 'dep', 'mic', 'dis']
entity_type_dict = {
            'dru':'药物',
            'bod':'身体',
            'pro':'医疗程序',
            'sym':'临床表现',
            'equ':'医疗设备',
            'ite':'医学检验项目',
            'dep':'科室',
            'mic':'微生物类',
            'dis':'疾病'
}
merged_list = merge_details(test_details)
get_results(merged_list, input_list, entity_type_dict, ent_extend_map, 'baichuan2_13B_zh')


Baichuan2-ACE05

In [3]:
with open('./results/details_baichuan2_13B_en.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/ace05/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/ace05/final_entity_extend_map_baichuan2_13B_en.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)


merged_list = merge_details(test_details)
get_results(merged_list, input_list, None, ent_extend_map, 'baichuan2_13B_en')

Alpaca

In [3]:
with open('./results/details_alpaca_33B_zh.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/cmeee/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/cmeee/final_entity_extend_map_alpaca_33B_zh.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)

# entity_type_list = ['dru', 'bod', 'pro', 'sym', 'equ', 'ite', 'dep', 'mic', 'dis']
entity_type_dict = {
            'dru':'药物',
            'bod':'身体',
            'pro':'医疗程序',
            'sym':'临床表现',
            'equ':'医疗设备',
            'ite':'医学检验项目',
            'dep':'科室',
            'mic':'微生物类',
            'dis':'疾病'
}
merged_list = merge_details(test_details)
get_results(merged_list, input_list, entity_type_dict, ent_extend_map, 'alpaca_33B_zh')


In [4]:
with open('./results/details_alpaca_33B_en.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/ace05/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/ace05/final_entity_extend_map_alpaca_33B_en.json','r',encoding='utf-8') as f:
    ent_extend_map = json.load(f)


merged_list = merge_details(test_details)
get_results(merged_list, input_list, None, ent_extend_map, 'alpaca_33B_en')